## Acoustic Integration - Room Size Estimation

http://localhost:8888/?token=sloth

In [ ]:
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt

import drjit as dr
import mitsuba as mi

from utils import drjit_turn_off_optimizations, shoebox_scene, plot_hist, dict_replace
from acoustic_params import EDC, T

plt.style.use('ggplot')

mi.set_variant('cuda_ad_acoustic')
mi.set_log_level(mi.LogLevel.Warn)

drjit_turn_off_optimizations(False)

In [ ]:
scattering = [0.6, 0.95]  #, 0.5, 0.75, 1.0]
absorption = [0.1, 0.25] #, 0.5, 0.75, 0.9]

absorption_mdim, scattering_mdim = np.meshgrid(absorption, scattering)
absorption,      scattering      = absorption_mdim.flatten(), scattering_mdim.flatten()

config = {
    "box_dim":     [25., 12., 7.],
    "mic_pos":     [10.,  6., 3.5],
    "speaker_pos": [15.,  6., 3.5],
    "speaker_radius": 0.5,

    "absorption": [(i + 1, a) for i, a in enumerate(absorption)],
    "scattering": [(i + 1, s) for i, s in enumerate(scattering)],

    "wav_bins":  scattering.shape[0], # x
    "time_bins": 2000,                # y
    "max_time":  2,

    "integrator": "acousticpath",
    # "integrator": "prb_acoustic",
    "max_depth": 50,
    "spp": 2**20,
}

In [ ]:
scene_dict = shoebox_scene(connected_cube=True, **config)
scene      = mi.load_dict(scene_dict)
time       = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)

params = mi.traverse(scene)
vertex_pos_ref = dr.unravel(mi.Point3f, params["shoebox.vertex_positions"])
display(params)

def scale_shoebox(vertex_ref, scale=1.0):
    params["shoebox.vertex_positions"] = dr.ravel(mi.Transform4f.scale(scale) @ vertex_ref)
    params.update()

In [ ]:
new_conf = dict_replace(config, "absorption", None)

scale_shoebox(vertex_pos_ref, scale=1.0)
hist = mi.render(scene) / config['spp']
plot_hist(dr.log(hist[:, :, 0]), time=time, **new_conf)

scale_shoebox(vertex_pos_ref, scale=1.5)
hist = mi.render(scene) / config['spp']
plot_hist(dr.log(hist[:, :, 0]), time=time, **new_conf)

In [ ]:
N = 10
scale_factors = dr.linspace(mi.Float, 1.0, 10.0, N)

results = np.zeros((N, config["wav_bins"]))

for i in trange(N):
    scale_shoebox(vertex_pos_ref, scale=scale_factors[i])
    hist  = np.array(mi.render(scene))[:, :, 0] / config["spp"]
    edc = EDC(hist, db=True)
    # plt.plot(edc)
    # plt.show()

    for j in range(config["wav_bins"]):
        results[i, j] = T(time, edc[:, j], dB_init=-5., dB_decay=20.)

In [ ]:
label = list(map(lambda x: f"$\\alpha = {x:.2f}$", absorption_mdim[0]))

fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(12, 4))

ax1.set_title(f"scattering = {scattering_mdim[0, 0]}")
ax1.plot(scale_factors, results[:, :2], label=label)

ax2.set_title(f"scattering = {scattering_mdim[1, 0]}")
ax2.plot(scale_factors, results[:, 2:], label=label)

ax1.set_xlabel('room size scaling factor')
ax2.set_xlabel('room size scaling factor')
ax1.set_ylabel('$T_{30}$')
plt.show()